## Api Rest python 

imports + config clés api

In [ ]:
import requests
import json
import os
#  Récupérer la météo d'une ville 
def get_weather(city='Paris'):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': '5d50259803abd2f048a70830f7f49ebd',
        'units': 'metric'
    }
    
    try:
        response = requests.get(url, params=params)
        return response.json()
    except:
        print(f" Erreur pour {city}")
        return None

Question 1 et  2 : Variable d'environement + script conditionnel

In [ ]:

#  Stockage dans une variable d'environnement
def store_weather(weather_data):
    if not weather_data:
        return None
        
    weather_main = weather_data['weather'][0]['main']
    os.environ['WEATHER_CONDITION'] = weather_main
    
    print(f" Météo stockée: {weather_main}")
    return weather_main

#  Script conditionnel selon la météo
def show_weather_message():
    weather = os.getenv('WEATHER_CONDITION')
    
    print(f"\n Condition météo: {weather}")
    
    if weather == 'Rain':
        print(" Il pleut ! ")
        
    elif weather == 'Clear':
        print(" Il fait beau ! ")
        
    elif weather == 'Clouds':
        print(" Temps nuageux ")
        
    elif weather == 'Snow':
        print(" Il neige !  ")
        
    elif weather == 'Thunderstorm':
        print(" Orage ! ")
        
    else:
        print(f" Météo: {weather}")

#  Programme principal
def main():
    print("=== SCRIPT MÉTÉO  ===")
    
    # Récupération des données
    print(" Récupération de la météo...")
    data = get_weather()
    
    if not data:
        print(" Impossible de récupérer la météo")
        return
    
    # Stockage dans l'environnement  
    print(" Stockage dans variable d'environnement...")
    store_weather(data)
    
    # Affichage du message conditionnel
    print(" Affichage du message:")
    show_weather_message()
    
    print(f"\n🔍 Variable: WEATHER_CONDITION = {os.getenv('WEATHER_CONDITION')}")

# Lancer le programme
if __name__ == "__main__":
    main()

Question 3: exploiter la donnéee feel_like

In [ ]:
#  Analyser feels_like vs température
def analyze_feels_like(weather_data):
    if not weather_data:
        return
    
    temp = weather_data['main']['temp']
    feels_like = weather_data['main']['feels_like']
    difference = feels_like - temp
    city = weather_data['name']
    
    print(f"\n {city}:")
    print(f"   Température: {temp}°C")
    print(f"   Ressenti: {feels_like}°C")
    print(f"   Différence: {difference:+.1f}°C")
    
    # Interprétation IA
    if abs(difference) < 1:
        print("    IA: Conditions confortables")
    elif difference > 2:
        print("    IA: Recommander hydratation")
    elif difference < -2:
        print("    IA: Recommander vêtements chauds")



question4 : enregistrement dans une collection   structuré

In [ ]:
#  Créer dataset JSON de 5 villes
def create_weather_dataset():
    cities = ['Paris', 'London', 'Tokyo', 'New York', 'Sydney']
    dataset = []
    
    print(" Collecte données 5 villes...")
    
    for city in cities:
        data = get_weather(city)
        if data:
            city_info = {
                'ville': data['name'],
                'temperature': data['main']['temp'],
                'humidite': data['main']['humidity'],
                'etat_ciel': data['weather'][0]['main'],
                'feels_like': data['main']['feels_like']
            }
            dataset.append(city_info)
            analyze_feels_like(data)
    
    return dataset

#  Sauvegarde et analyser le dataset
def save_dataset(dataset):
    filename = 'weather_dataset.json'
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, indent=2, ensure_ascii=False)
    
    print(f"\n Dataset sauvegardé: {filename}")
    print("\n DATASET JSON:")
    print(json.dumps(dataset, indent=2, ensure_ascii=False))
    
    print("\n TRANSFORMATION POUR ML:")
    print(" Features: température, humidité → Prédire feels_like")
    print(" Preprocessing: Normalisation + One-hot encoding")
    print(" Modèles: Régression pour prédiction température")

#  Programme principal
def main():
    print("=== ANALYSE FEELS_LIKE + DATASET ===")
    
    # Analyse d'une ville
    print("1. Analyse feels_like Paris...")
    data = get_weather('Paris')
    if data:
        os.environ['WEATHER_CONDITION'] = data['weather'][0]['main']
        analyze_feels_like(data)
    
    # Créer dataset 5 villes
    print("\n2. Création dataset 5 villes...")
    dataset = create_weather_dataset()
    
    # Sauvegarde
    print("\n3. Sauvegarde et analyse ML...")
    save_dataset(dataset)
    
    print(f"\n Variable: WEATHER_CONDITION = {os.getenv('WEATHER_CONDITION')}")

# Lancer le programme
if __name__ == "__main__":
    main()

question 5 : verification température critique

In [ ]:
#  Vérification température critique
def check_temperature_alert(weather_data):
    if not weather_data:
        return
    
    temp = weather_data['main']['temp']
    city = weather_data['name']
    
    print(f"\n {city}: {temp}°C")
    
    # Alerte si > 40°C
    if temp > 40:
        print(" ALERTE CANICULE!")
        print("    Buvez beaucoup d'eau")
        print("    Restez au frais")
        
        # Sauvegarder alerte
        alert = {
            'ville': city,
            'temperature': temp,
            'type': 'CANICULE',
            'heure': datetime.now().strftime('%H:%M:%S')
        }
        
        with open('alerte.json', 'w') as f:
            json.dump(alert, f, indent=2)
        
        print("    Alerte sauvegardée!")
        
    else:
        print("    Température normale")

In [ ]:
def create_dataset():
    cities = ['Paris', 'London', 'Tokyo', 'New York', 'Sydney']
    dataset = []
    
    for city in cities:
        data = get_weather(city)
        if data:
            city_info = {
                'ville': city,
                'temperature': data['main']['temp'],
                'humidite': data['main']['humidity'],
                'etat_ciel': data['weather'][0]['main']
            }
            dataset.append(city_info)
            check_temperature_alert(data)
    
    # Sauvegarder dataset
    with open('dataset.json', 'w') as f:
        json.dump(dataset, f, indent=2)
    
    print(f"\n Dataset sauvegardé: {len(dataset)} villes")
    return dataset

#  Programme principal
def main():
    print("=== MÉTÉO + ALERTES ===")
    
    # Test Paris
    data = get_weather('Paris')
    if data:
        os.environ['WEATHER_CONDITION'] = data['weather'][0]['main']
        check_temperature_alert(data)
    
    # Dataset 5 villes
    print("\n Collecte 5 villes...")
    dataset = create_dataset()
    
    print(f"\n Météo stockée: {os.getenv('WEATHER_CONDITION')}")


# Lancer
if __name__ == "__main__":
    main()
   

question 7 : comparer 2 villes 

In [ ]:

#  Comparer deux villes
def compare_cities(city1, city2):
    print(f"\n🌍 COMPARAISON: {city1} vs {city2}")
    print("=" * 50)
    
    # Récupérer données
    data1 = get_weather(city1)
    data2 = get_weather(city2)
    
    if not data1 or not data2:
        print(" Impossible de comparer")
        return None
    
    # Extraire indicateurs
    temp1 = data1['main']['temp']
    temp2 = data2['main']['temp']
    humidity1 = data1['main']['humidity']
    humidity2 = data2['main']['humidity']
    pressure1 = data1['main']['pressure']
    pressure2 = data2['main']['pressure']
    
    # Affichage
    print(f" TEMPÉRATURE:")
    print(f"   {city1}: {temp1}°C")
    print(f"   {city2}: {temp2}°C")
    print(f"   → Plus chaud: {city1 if temp1 > temp2 else city2}")
    
    print(f"\n HUMIDITÉ:")
    print(f"   {city1}: {humidity1}%")
    print(f"   {city2}: {humidity2}%")
    
    print(f"\n PRESSION:")
    print(f"   {city1}: {pressure1} hPa")
    print(f"   {city2}: {pressure2} hPa")
    
    # Retourner données
    return {
        'ville1': city1,
        'ville2': city2,
        'temperature': {'ville1': temp1, 'ville2': temp2, 'diff': abs(temp1 - temp2)},
        'humidite': {'ville1': humidity1, 'ville2': humidity2, 'diff': abs(humidity1 - humidity2)}
    }

# 
#  Programme principal
def main():
    print("=== COMPARATEUR MÉTÉO ===")
    
    # Comparaison Alger vs Montréal
    print("\n1. Comparaison Alger vs Montréal...")
    comp1 = compare_cities('Alger', 'Montréal')
    
    # Stocker dans env
    if comp1:
        os.environ['LAST_COMPARISON'] = f"{comp1['ville1']}_vs_{comp1['ville2']}"
        print(f"\n Dernière comparaison: {os.getenv('LAST_COMPARISON')}")

# Lancer
if __name__ == "__main__":
    main()